In [ ]:
import siibra
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0a14')
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
from nilearn import plotting

## Select a probability map of a cortical brain region from the Julich-Brain atlas

We start this tutorial by selecting a cortical brain region from the Julich-Brain cytoarchitectonic atlas. 

In [ ]:
# we will search for the region by keywords
regionspec = '4p'
hemisphere = 'right'

In [ ]:
# Get the parcellation, its map in MNI space
julichbrain = siibra.parcellations.get('julich 3')
region = julichbrain.get_region(f'{regionspec} {hemisphere}')

In [ ]:
# get access to the corresponding probabilistc maps in MNI space, 
# and fetch the one for the selected region.
julich_pmaps = julichbrain.get_map(space='mni152', maptype='statistical')
pmap = julich_pmaps.fetch(region=region)

In [ ]:
# The result is a NIfTI image object.
# We can plot it with common neuroscience libraries; 
# here we juse nilearn's wonderful 'plotting' module.
plotting.plot_glass_brain(pmap, cmap='viridis', title=region.name)

## Find a 1-micron section intersecting the brain region

We can use region maps to query spatial features.
Here we search for 1 micron sections of the BigBrain which intersect with the region map.
Note that siibra automatically resovles the mismatch between the template spaces:
It will warp the bounding boxes of BigBrain sections to MNI space for the query.

In [ ]:
# find 1 micron section in bigbrain space which overlaps with the region
pmap_volume = siibra.volumes.from_nifti(pmap, space='mni152', name=region.name)
sections = siibra.features.get(pmap_volume, siibra.features.cellular.CellbodyStainedSection)
section = sections[len(sections)//2]

## Intersect surface meshes with the image plane

We are interested in cortical structure. Here it is very hepful to project the cortical layer surfaces to the image section. This is a 3D to 2D projection, leading to sets of (possibly split) contours of the layer surfaces.

In [ ]:
# access cortical layer maps in BigBrain space
layers = siibra.parcellations.get('cortical layers')
layermap = layers.get_map(space='bigbrain')

# Derive a 3D plan from the 1 micron section.
# Note that the 1 micron section is a 3D image volume,
# however its y-dimension is flat. 
# Siibra will create a plane from the smallest image dimension.
# The plane is defined in the physical brain reference space.
imgplane = siibra.experimental.Plane3D.from_image(section)
print(f"Extracted a plane in {imgplane.space} space, with normal {imgplane.normal}.")

# Intersect all meshes in the parcellation map with this plane
# to obtain layer contours. 
# The contours are represented as ordered sets of points.
layer_contours = {
    l: imgplane.intersect_mesh(layermap.fetch(region=l, format='mesh'))
    for l in layermap.regions
}

for layername, contours in layer_contours.items():
    print(
        f" - {layername+':':30} {len(contours)} intersected contours."
    )

In [ ]:
# plot the layer contours with a lower resolution version of the image section.
# We do this with matplotlib, using the actual pixel space of the image,
# so the image is not properly oriented here.

# fetch a 0.8mm version
im_low_res = section.fetch(resolution_mm=0.8)

plt.figure()
plt.imshow(im_low_res.get_fdata().squeeze(), cmap='gray', vmin=0, vmax=2**16)
plt.axis('off')

# siibra's map objects typically provide colormaps, let's use them!
layercolors = {
    l:layermap.get_colormap().colors[layermap.get_index(l).label]
    for l in layermap.regions
}

# plot the contours
for layername, contour_list in layer_contours.items():
    for contours in contour_list:
        phys2pix = np.linalg.inv(im_low_res.affine)
        pixels = contours.transform(phys2pix, space=None).homogeneous
        plt.plot(pixels[:, 2], pixels[:, 0], '-', color=layercolors[layername]) 

## Find a closeby cortical profile in the brain region

We want to find a cortical profile which is likely in the selected brain region, and also as close as possible to the image section.

We do so by looking up the values of layer IV contour points in the region map, and selecting the point with highest probability.

Then we find the "column" of corresponding surface vertices in the cortical layer maps closest to the contour point.

In [ ]:
# find the most probable contour point in the layer 4 surface
l4layer = layers.get_region(f"4 {hemisphere}")

# collect all layer IV contour points into one set
l4points = siibra.PointSet(
    [p for contour in layer_contours[l4layer.name] for p in contour], 
    space='bigbrain'
)

# use the probabilistic map volume to lookup their values 
probs = pmap_volume.evaluate_points(l4points)

# get the highest ranked point
l4point = l4points[probs.argmax()]

# siibra has an experimental class for querying cortical profiles
profile = siibra.experimental.CorticalProfileSampler().query(l4point)

In [ ]:
# plot the image crop around the brain region with layers and the profile
crop_voi = section.get_boundingbox().intersection(pmap_volume)
crop = section.fetch(voi=crop_voi, resolution_mm=0.2)
phys2pix = np.linalg.inv(crop.affine)

# plot the image
plt.figure()
plt.imshow(crop.get_fdata().squeeze(), cmap='gray', vmin=0, vmax=2**16)

# plot the contour segments
for layername, contours in layer_contours.items():
    for contour in contours:
        for segment in contour.crop(crop_voi):
            pixels = segment.transform(phys2pix, space=None).homogeneous
            plt.plot(pixels[:, 2], pixels[:, 0], '-', ms=4, color=layercolors[layername])

# plot the profile points
for p in profile:
    x, y, z = p.transform(phys2pix, space=None)
    plt.plot(z, x, 'r.', ms=10)
    
plt.axis('off')

## Define the cortical image patch including the profile

Now extract a cortical patch from the section which is centered around the layer 4 point and approximately rotated orthogonal to the brain surface.

siibra's Plane3D class can extract a rotated patch definition from a set of points, which are projected onto the plane.
The rotation is derived from the principal axes of the projected points.
The patch can then extract image data in the new orientation from the original image volume.

Since this function is not aware of the special ordering of the profile points however, the patch might be extracted upside down. 
We can detect this using a bit of a hack, by checking to which of the corner points the inner surface point is closest, and flip if required.

In [ ]:
patch_canvas = imgplane.get_enclosing_patch(profile)
i = np.argmin(
    np.linalg.norm((patch_canvas.corners.coordinates - profile[0].coordinate), axis=1)
)
if i in [0, 3]:
    patch_canvas.flip()
    
# now fetch the image data
patch = patch_canvas.extract_volume(section, resolution_mm=0.01)

In [ ]:
# plot the patch individually, and in 3D context.
# Here we first use nilearn plotting functions with the resulting NIfTI files,
# so the extracted patch is still shown in its anatomical orientation.
view = plotting.plot_img(patch.fetch(), cmap='gray', display_mode='y', annotate=False, title=f"Rotated patch")
template = siibra.get_template("bigbrain")
bigbrain_lowres = template.fetch(resolution_mm=0.8)
view = plotting.plot_img(patch.fetch(), bg_img=bigbrain_lowres, cmap='gray', title="Rotated patch in reference space")

In [ ]:
# to verify that the underlying pixels have been resampled
# into local cortical orientation, we display the underlying image
# using matplotlib.

plt.figure()

# access the underlying image data
patchimg = patch.fetch()
phys2vox = np.linalg.inv(patchimg.affine)
patchdata = patchimg.get_fdata().squeeze()

# plot the pure image array
plt.imshow(patchdata, cmap='gray', vmin=0, vmax=2**16)
plt.axis('off')


# overlay fragments of intersected layer surface contours.
for layername, contours in layer_contours.items():
    for contour in contours:
        for s in contour.crop(patch.get_boundingbox()):
            pixels = s.transform(phys2vox, space=None).homogeneous
            plt.plot(pixels[:, 2], pixels[:, 0], '-', ms=4, color=layercolors[layername])

# let's show the title of the extracted patch for fun...
import textwrap
plt.title("\n".join(textwrap.wrap(patch.name, 50)))

plt.tight_layout()